在进行文本相似度分析的步骤：
 1. 文本预处理 ： 对于拿到的文本资料进行分词，去除停用词，词性标注，归一化处理等。
 2. 文本特征提取 ： 
  * 词袋模型
  * TF-IDF
  * 词嵌入
 3. 计算相似度：
  * 计算余弦相似度
  * 欧氏距离
  * Jaccard

### 单词的表示
要让计算机理解单词的含义，有以下方法：
 * 基于计数的方法
 * 基于推理的方法(也称基于分布式表示)

#### 使用 TF-IDF 和余弦相似度计算文本之间的相似度
TF-IDF 是一种常用的文本特征提取方法，用于衡量一个词在文档集合中的重要性。TF-IDF 结合了词频（Term Frequency, TF）和逆文档频率（Inverse Document Frequency, IDF）两个概念。   
词频（TF）：表示一个词在文档中出现的频率。    
逆文档频率（IDF）：表示一个词在所有文档中出现的稀有程度。    
公式为: ```TF-IDF(t, d)=TF(t, d) * IDF(t)```
其中： ```TF(t, d) = 词 t 在文档 d 中出现的次数/文档 d 中总词数```   
    ```IDF(t) = log(语料库中的文档总数/包含词 t 的文档总数)```


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

text1 = ['I like to play football.']
text2 = ['I love playing football.']

text = text1 + text2
# 初始化 TF-IDF 向量化器
vectorizer = TfidfVectorizer()

# 将文本数据转换为 TF-IDF 特征向量
tf_vecs = vectorizer.fit_transform(text)

tf_vec1 = tf_vecs[0:1]
tf_vec2 = tf_vecs[1:2]

# 计算余弦相似度矩阵
cosine_sim = cosine_similarity(tf_vec1, tf_vec2)
print(f"Similarity:{cosine_sim}")

Similarity:[[0.17077611]]


In [8]:
'''TF-IDF 使用的是基于计数的方法'''
print(f"tf_vec1: {tf_vec1.toarray()}")
print(f"tf_vec2: {tf_vec2.toarray()}")

tf_vec1: [[0.37997836 0.53404633 0.         0.53404633 0.         0.53404633]]
tf_vec2: [[0.44943642 0.         0.6316672  0.         0.6316672  0.        ]]


### 基于词嵌入的计算文本相似度
词嵌入主要是将文本中的单词表示为一个向量，如果文本```text```有10个单词，我们将每个单词对应的向量维度定义为300维，那么这句话可以表示形状为为(10, 300)的此矩阵。   
词嵌入通过训练神经网络模型在高维空间中为每个单词生成一个向量，这些向量可以捕捉单词的语义和上下文信息。词嵌入的模型主要有 ```CBOW```、```Glove```、```Skip-gram```, 使用词嵌入可以使单词的每个维度都代表相应的含义。

In [19]:
'''  
使用已经训练好的预训练的 Glove 模型来进行相似度检验
'''
from gensim.models import KeyedVectors

# 指定 GloVe 预训练向量文件的路径
glove_file = r'D:\repo\glove.6B.100d.txt'

# 使用 Gensim 加载 GloVe 预训练向量
model = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)


In [22]:
'''  
使用词嵌入求文本相似度，把得到的单词对应的词嵌入向量相加求平均
'''
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# 预处理文本
def preprocessing(text):
    return text.lower().split()

def avg_vec(text, model):
    words = preprocessing(text)
    avg_vec_ = np.mean([model[word] for word in words if word in model], axis=0) # 每个单词对应一个sample, 求的是 sample 平均
    return avg_vec_

# 文本
text1 = 'I like to play football.'
text2 = 'I love playing football.'

vec1 = avg_vec(text1, model).reshape(1, -1)
vec2 = avg_vec(text2, model).reshape(1, -1)

# 计算相似度
cos_sim = cosine_similarity(vec1, vec2)
print(f"Similarity: {cos_sim}")


Similarity: [[0.906745]]


可以看到使用词嵌入，两句话的相似度很高。

如果给定的文本文本相对较长，我们可以先用深度学习模型对其进行文本提取， 然后对于提取的文本摘要进行相似度计算。